# Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

Steps:

1. Problem definition
2. Manipulating data
3. Evaluation
4. Identifying Features
5. Modelling
6. Experimentation

## 1. Problem Definition

